<a href="https://colab.research.google.com/github/whitehatdevelop/exercise_ml/blob/master/exercise_pre_procesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CONEXIÓN A DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#INGRESANDO A CARPETA CONTENEDORA DE ARCHIVOS EN DRIVE "ml_database"
%cd '/content/drive/My Drive/Colab Notebooks/ml_database'

/content/drive/My Drive/Colab Notebooks/ml_database


In [ ]:
#IMPORTACIÓN DE LIBRERÍAS
import pandas as pd #IMPORTACIÓN DE LIBRERIA - PANDAS
import numpy as np

import seaborn as sn #IMPORTACIÓN DE LIBRERIA - SEABORN
import matplotlib.pyplot as plt #IMPORTACIÓN DE LIBRERIA - MATPLOT

%matplotlib inline

In [ ]:
#CARGA DE ARCHIVO .CSV 
dataset = pd.read_csv('Dataset_AfiliacionDebitoAutom.csv',
                      sep=',', #DEFINIENDO SEPARADOR ',' PROPIO DE .CSV
                      encoding='utf-8', #CODIFICACIÓN UTF-8
                      na_values=' ')

In [ ]:
#VISUALIZACIÓN DE ENCABEZADOS DE LAS COLUMNAS DEL DATAFRAME COMO UNA LISTA
dataset.columns.values.tolist()

['N_ACTIVE_LINES',
 'N_SUSPENDED_LINES',
 'DEBT_CAPACITY',
 'AGE',
 'INCOME',
 'OLDEST_LINE_MONTHS',
 'SAME_BILLING_3M',
 'DIGITAL_BILL',
 'MB_3G_USE',
 'MB_4G_USE',
 'VOICE_IN_3M',
 'VOICE_OUT_3M',
 'SUBSCRIPTION_FLAG']

In [ ]:
#VISUALIZACIÓN DE LOS 5 PRIMEROS DATOS DEL DATAFRAME
dataset.head(5)  

,N_ACTIVE_LINES,N_SUSPENDED_LINES,DEBT_CAPACITY,AGE,INCOME,OLDEST_LINE_MONTHS,SAME_BILLING_3M,DIGITAL_BILL,MB_3G_USE,MB_4G_USE,VOICE_IN_3M,VOICE_OUT_3M,SUBSCRIPTION_FLAG
0,2,0,570,35.0,173.699997,27,1,1,0.0,0.0,0.7999,0.7999,0.0
1,1,0,80,39.0,79.000056,12,0,1,0.0,0.0,0.0000,0.0000,0.0
2,1,0,120,35.0,29.900001,5,1,1,4.9,2077.8,60.5667,81.6664,0.0
3,0,1,0,34.0,62.900000,14,1,1,9.8,17.9,101.5669,418.7003,1.0
4,1,0,186,35.0,65.899460,39,0,1,50.3,9547.0,260.8001,917.3007,1.0


In [ ]:
#VISUALIZACIÓN DE DATOS ESTADÍSTICOS DEL DATAFRAME
dataset.describe()

,N_ACTIVE_LINES,N_SUSPENDED_LINES,DEBT_CAPACITY,AGE,INCOME,OLDEST_LINE_MONTHS,SAME_BILLING_3M,DIGITAL_BILL,MB_3G_USE,MB_4G_USE,VOICE_IN_3M,VOICE_OUT_3M,SUBSCRIPTION_FLAG
count,101.000000,101.000000,101.000000,95.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,99.000000,101.000000,100.000000
mean,1.059406,0.138614,172.861386,36.463158,68.047274,17.237624,0.633663,0.801980,731.056436,7166.463366,252.033448,620.201870,0.600000
std,0.580031,0.374959,126.132948,9.120716,43.298222,15.643624,0.484206,0.400495,1781.948329,10819.287102,476.432244,724.473191,0.492366
min,0.000000,0.000000,0.000000,24.000000,18.978000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,90.000000,28.000000,41.900002,6.000000,0.000000,1.000000,21.100000,377.200000,48.491850,154.900000,0.000000
50%,1.000000,0.000000,160.000000,35.000000,62.900000,11.000000,1.000000,1.000000,167.800000,3389.500000,121.950200,418.700300,1.000000
75%,1.000000,0.000000,220.000000,43.000000,79.000056,26.000000,1.000000,1.000000,604.600000,9061.300000,306.091850,871.984500,1.000000
max,3.000000,2.000000,570.000000,57.000000,309.599459,80.000000,1.000000,1.000000,14734.000000,56071.000000,4296.182900,5014.400300,1.000000


In [ ]:
#VERIFICANDO EL NÚMERO DE DATOS NULLABLES POR COLUMNA
dataset.isnull().sum(axis=0)

N_ACTIVE_LINES        0
N_SUSPENDED_LINES     0
DEBT_CAPACITY         0
AGE                   6
INCOME                0
OLDEST_LINE_MONTHS    0
SAME_BILLING_3M       0
DIGITAL_BILL          0
MB_3G_USE             0
MB_4G_USE             0
VOICE_IN_3M           2
VOICE_OUT_3M          0
SUBSCRIPTION_FLAG     1
dtype: int64

In [ ]:
#INSERTANDO MEDIANA EN VALORES NULLABLES
dataset = dataset.fillna({'AGE':dataset['AGE'].median()}) #REEMPLAZANDO DENTRO DE COLUMNA 'AGE'
dataset = dataset.fillna({'VOICE_IN_3M':dataset['VOICE_IN_3M'].median()}) #REEMPLAZANDO DENTRO DE COLUMNA 'VOICE_IN_3M'
dataset = dataset.fillna({'SUBSCRIPTION_FLAG':dataset['SUBSCRIPTION_FLAG'].median()}) #REEMPLAZANDO DENTRO DE COLUMNA 'SUBSCRIPTION_FLAG'

In [ ]:
#VERIFICANDO QUE EL NÚMERO DE DATOS NULLABLES POR COLUMNA AHORA SEAN 0
dataset.isnull().sum(axis=0)

N_ACTIVE_LINES        0
N_SUSPENDED_LINES     0
DEBT_CAPACITY         0
AGE                   0
INCOME                0
OLDEST_LINE_MONTHS    0
SAME_BILLING_3M       0
DIGITAL_BILL          0
MB_3G_USE             0
MB_4G_USE             0
VOICE_IN_3M           0
VOICE_OUT_3M          0
SUBSCRIPTION_FLAG     0
dtype: int64

In [ ]:
#CONTEO DE DATOS EN VARIABLE OBJETIVO 'SUBSCRIPTION_FLAG'
dataset.SUBSCRIPTION_FLAG.value_counts() #dataset.groupby('SUBSCRIPTION_FLAG').size()

1.0    61
0.0    40
Name: SUBSCRIPTION_FLAG, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split 
columns = dataset.columns.tolist() #COLUMNAS DE DATASET CONVERTIDAS EN UNA LISTA
columns = [x for x in columns if x not in ["SUBSCRIPTION_FLAG"]]  #DEFINIENDO UNA LIST COMPREHENSION

X = dataset[columns] #VARIABLES REGRESORAS
Y = dataset['SUBSCRIPTION_FLAG'] #VARIABLE RESPUESTA
X_train, X_test, y_train, y_test = train_test_split(X,Y,
                                                    test_size=.3, #PORCENTAJE PARA TESTING
                                                    random_state=22) #FIJAR SEMILLA PARA OBTENER EL MISMO RESULTADO AL VOLVER A EJECUTAR

In [ ]:
target = dataset['SUBSCRIPTION_FLAG']
target

0      0.0
1      0.0
2      0.0
3      1.0
4      1.0
      ... 
96     1.0
97     1.0
98     1.0
99     1.0
100    1.0
Name: SUBSCRIPTION_FLAG, Length: 101, dtype: float64

In [ ]:
dataset.SUBSCRIPTION_FLAG.value_counts() #VISUALIZANDO CANTIDAD DENTRO DE SUBSCRIPTION_FLAG AGRUPADAS POR LOS VALORES PERTENECIENTES A LA COLUMNA

1.0    61
0.0    40
Name: SUBSCRIPTION_FLAG, dtype: int64

In [ ]:
target_count = dataset.SUBSCRIPTION_FLAG.value_counts() #CONTEO DE SUBSCRIPTION_FLAG 0 | 1
count_unaffiliate = target_count[0] #CANTIDAD DE "NO AFILIADOS"
count_affiliate = target_count[1] #CANTIDAD DE "NO AFILIADOS"

count_affiliate,count_unaffiliate #VISUALIZANDO CANTIDADES

(61, 40)

In [ ]:
#SEPARAR EN DOS DATAFRAMES EL DATASET 
dataset_unaffiliate = dataset[dataset['SUBSCRIPTION_FLAG'] == 0]
dataset_affiliate = dataset[dataset['SUBSCRIPTION_FLAG'] == 1]

In [ ]:
#APLICANDO SOBRE-MUESTREO A LOS DATOS
dataset_affiliate_over = dataset_affiliate.sample(count_unaffiliate, replace=True) #NUEVO DATAFRAME DE "AFILIADOS" BASADO EN UNA MUESTRA DEL DATAFRAME DE "AFILIADO" DEL TAMAÑO DE LOS "NO AFILIADOS"

df_test_over = pd.concat([dataset_unaffiliate, dataset_affiliate_over], axis=0) #CONCATENANDO EL DATAFRAME "NO AFILIADO" CON EL NUEVO DATAFRAME "AFILIADOS" DEL SOBREMUESTREO
df_test_over.groupby('SUBSCRIPTION_FLAG').size() #EL TAMAÑO DEL DATASET QUEDA BALANCEADO AL NÚMERO DE "NO AFILIADOS"

SUBSCRIPTION_FLAG
0.0    40
1.0    40
dtype: int64